
# Barbero durmiente

Para tratar el problema se  ha implementado una clase que simula la Barberria con los procesos que se ejecutan dentro de ella.
Se proporciona el esqueleto de la implementación de los procesos para que se marquen las zonas de exclusión mutua, de espera o de sincronazación. 
En el constructor de la Barberia se inicilizan todos los cerrojos, semaforos, condiciones y variables que se necesitan para la implementación.

In [14]:
import threading
import time
import sys

class Barberia():
    def __init__(self):
        threading.Thread.__init__(self)
        ############################################################
        ## Control de los clientes
        ############################################################
        # Solo puedo tener 3 clientes en la sala de espera. Lo hago con un semáforo.
        # Ojo, estar en la sala != de estar en la silla.
        self.entrarEnSala = threading.Semaphore(3)
        #El numero de clientes en la sala se incializa a cero. 
        self.clientesEnSala = 0
        # Garantiza exclusion para modificacion de variables de control de sala o acceso a secciones.
        # Asi evito que cambie el numero de clientes sin poder controlarlo.
        self.exclusionMutuaSala=threading.Semaphore()
        ############################################################
        ## Control del barbero
        ############################################################
        # El sillon será un cerrojo, así si te sientas no te puedes levantar. No queremos que el cliente se vaya a medio corte de pelo.
        self.sillon = threading.Condition()
        
        # Podemos tener al barbero ocupado o dormido. De base, el barbero estará libre (ocupado = false)  y dormido
        self.barbero_ocupado=False
        #puede poner al barbero a dormir y lo uso como condicion para el wait
        self.barbero_dormido=threading.Condition()
        

    def actividadBarbero(self):
        while True:
            ### INCLUYE EL CÓDIGO NECESARIO PARA PROTOCOLO INICIAL   ###
            ### consultar en exclusion mutua los clientes qUe hay en la sala###
            ### si no hay clientes esperando se pone a dormir hasta que le despierten ####
            self.exclusionMutuaSala.acquire() 
            if self.clientesEnSala > 0:
                self.exclusionMutuaSala.release()
                print("El barbero da paso a un cliente\n")
                ###  INTRODUCE EL CÓDIGO PARA CAMBIAR EL ESTADO DEL BARBERO ###
                print("El barbero corta a un cliente\n")
                time.sleep(5)
                self.barbero_ocupado = False
                with self.sillon:  
                    self.sillon.notify()
            else:
                self.exclusionMutuaSala.release()
                self.barbero_ocupado=False
                with self.barbero_dormido:  
                    print('El barbero esta dormido \n')
                    self.barbero_dormido.wait()

            

    def actividadCliente(self): 
            print("El cliente "+threading.currentThread().name+" llega a la puerta de la barbería \n")
            #Si no hay 3 esperando, puedo entrar. 
            if self.clientesEnSala < 3: 
                print("El cliente "+threading.currentThread().getName()+" entra en la sala\n")
                # Aviso al semaforo de que hay una persoan mas en la sala y la meto en la lista
                self.entrarEnSala.acquire()
                self.clientesEnSala +=1 
            else:
                #Aqui podria poner un bucle y cola para que espern en la puerta o reintentar la entrada.
                print("El cliente "+threading.currentThread().getName()+" se marcha por estar lleno \n")
                # Si el cliente se va, cierro su hilo
                sys.exit()
            #Para que no haya varias personas consultando al barbero
            self.exclusionMutuaSala.acquire() 
            # Si el barbero está ocupado, quiere decir que también está despierto y trabajando. Asi que se pone a esperar al sillon
            if self.barbero_ocupado:
                print ("El cliente "+threading.currentThread().name+" espera\n")
                self.exclusionMutuaSala.release()
                with self.sillon:  
                    self.sillon.wait()
                time.sleep(3)
                print ("Esta en el sillon  "+threading.currentThread().name+"\n")
                self.barbero_ocupado = True
            else:
                # Sabemos que si el barbero no esta ocupado, esta dormido, asi que lo despierta y se sienta en el sillon
                with self.barbero_dormido:  
                    print("El cliente "+threading.currentThread().getName()+" despierta al barbero\n")
                    self.barbero_dormido.notify()
                # Quito al cliente de la sala porque esta en el sillon
                print ("Esta en el sillon  "+threading.currentThread().name+"\n")
                time.sleep(3)
                self.barbero_ocupado = True
                self.exclusionMutuaSala.release() 
           
            print ("El cliente "+threading.currentThread().name+" abandona la barbería\n")
            self.clientesEnSala -=1 
            self.entrarEnSala.release()
            

if __name__=="__main__":
        barberia= Barberia()                 
        ### INCLUYE EL CÓDIGO NECESARIO PARA CREAR E INICIALIZAR LOS HILOS  ###
        barbero =  threading.Thread(target = barberia.actividadBarbero)
        cliente1 = threading.Thread(target = barberia.actividadCliente, name="Adan") 
        cliente2 = threading.Thread(target = barberia.actividadCliente, name="Pedro") 
        cliente3 = threading.Thread(target = barberia.actividadCliente, name="Juan")
        cliente4 = threading.Thread(target = barberia.actividadCliente, name="Lucas") 
        cliente5 = threading.Thread(target = barberia.actividadCliente, name="Marilo") 


        barbero.start()
        cliente1.start()
        cliente2.start()
        cliente3.start()
        cliente4.start()
        cliente5.start()








El barbero esta dormido 

El cliente Adan llaga a la puerta de la barbería 

El cliente Adan entra en la sala

El cliente Adan despierta al barbero

Esta en el sillon  Adan

El cliente Pedro llaga a la puerta de la barbería 

El cliente Juan llaga a la puerta de la barbería 
El cliente Pedro entra en la sala


El cliente Juan entra en la sala

El cliente Lucas llaga a la puerta de la barbería 

El cliente Marilo llaga a la puerta de la barbería 
El cliente Lucas se marcha por estar lleno 

El cliente Marilo se marcha por estar lleno 


El cliente Adan abandona la barbería
El barbero da paso a un cliente
El cliente Juan espera


El barbero corta a un cliente

El cliente Pedro espera


El barbero da paso a un cliente

El barbero corta a un cliente

Esta en el sillon  Juan

El cliente Juan abandona la barbería

El barbero da paso a un cliente

El barbero corta a un cliente

Esta en el sillon  Pedro

El cliente Pedro abandona la barbería

El barbero esta dormido 

